###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de https://microdatos.dane.gov.co/index.php/catalog/643/get_microdata y corresponden a datos cuya fuente primaria es el Departamento Administrativo Nacional de Estadisticas - DANE

In [1]:
import os
from pathlib import Path
import pandas as pd
import zipfile
import urllib.request

### Descarga de set de datos y carga como df

Contiene el resultado de la descarga de 33 conjuntos de datos, uno por cada departamento más Bogotá, que contiene los microdatos de los viviendas del territorio nacional del  censo de población y vivienda realizado en 2018.

El diccionario de datos de estos dataset puede consultarse en: https://microdatos.dane.gov.co/index.php/catalog/643/data-dictionary/F8?file_name=VIVENDAS

In [2]:
# Funcion para descargar, extraer y cargar datos del censo 2018
def load_census_data(department_name, url, file_prefix):
    """
    Descarga y extrae datos del Censo 2018 de un departamento desde una URL.

    Descarga un archivo .zip, extrae el subarchivo "_CSV", y carga el CSV que coincide con `file_prefix`.

    Args:
        department_name (str): Nombre del departamento.
        url (str): URL de descarga del archivo .zip.
        file_prefix (str): Prefijo del CSV a buscar (viviendas, viviendas, personas).

    Returns:
        pd.DataFrame or None: DataFrame con los datos del CSV o None si no se encuentra.

    El nombre del archivo CSV se forma con:
        f"{file_prefix}_{department_name[-2:]}.CSV"
    """

    # Ruta y URL del archivo zip a descargar
    zip_path = Path(f"datasets/{department_name}.zip")

    # Verificar si el archivo .zip ya existe, si no, descargarlo
    if not zip_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        urllib.request.urlretrieve(url, zip_path)

    # Extraer el archivo .zip descargado
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path="datasets")

    # Identificar dinámicamente la carpeta extraída
    extracted_folder = None
    for folder in Path("datasets").iterdir():
        if folder.is_dir() and department_name.split('_')[0] in folder.stem:
            extracted_folder = folder
            break

    if not extracted_folder:
        print(f"No se pudo encontrar la carpeta extraída para {department_name}")
        return None

        # Identificar el archivo .zip con sufijo "_CSV" y extraer su contenido
    csv_zip_path = None
    for item in extracted_folder.iterdir():
        if item.is_file() and item.suffix == '.zip' and "_CSV" in item.stem:
            csv_zip_path = item
            break

    # Si se encuentra el archivo _CSV, lo descomprimimos
    if csv_zip_path:
        with zipfile.ZipFile(csv_zip_path, 'r') as csv_zip_ref:
            csv_zip_ref.extractall(path=extracted_folder / "CSV")

    # Definir la ruta donde se descomprimieron los archivos CSV
    csv_folder = extracted_folder / "CSV"

    # Cargar el archivo CSV específico para el tipo de información solicitado
    file_to_load = f"{file_prefix}_{department_name[:2]}.CSV"
    csv_file_path = csv_folder / file_to_load
    if csv_file_path.is_file():
        return pd.read_csv(csv_file_path)
    else:
        print(f"Archivo no encontrado para {department_name}")
        return None


In [3]:
# Función para procesar una lista de URLs y concatenar los archivos
def process_multiple_departments(urls, file_prefix):
    """
    Procesa múltiples departamentos y concatena los datos en un único DataFrame.

    Itera sobre una lista de URLs y departamentos, usando `load_census_data` para cargar los CSVs
    y los concatena en un solo DataFrame.

    Args:
        urls (list of tuple): Lista de tuplas (nombre del departamento, URL).
        file_prefix (str): Prefijo del CSV a buscar (viviendas, viviendas, personas).

    Returns:
        pd.DataFrame: DataFrame con los datos combinados de todos los departamentos.
    """

    all_data = []
    for department_name, url in urls:
        df = load_census_data(department_name, url, file_prefix)
        if df is not None:
            all_data.append(df)

    # Concatenar todos los DataFrames en uno solo
    return pd.concat(all_data, ignore_index=True)


In [ ]:
# Lista de departamentos y URLs
departments_urls = [
     ("05_Antioquia", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12143"),
     ("08_Atlantico", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12144"),
     ("11_Bogota", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12145"),
     ("13_Bolivar", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12146"),
     ("15_Boyaca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12147"),
     ("17_Caldas", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12148"),
     ("18_Caqueta", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12149"),
     ("19_Cauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12150"),
     ("20_Cesar", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12151"),
     ("23_Cordoba", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12152"),
     ("25_Cundinamarca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12153"),
     ("27_Choco", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12154"),
     ("41_Huila", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12155"),
     ("44_La Guajira", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12156"),
     ("47_Magdalena", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12157"),
     ("50_Meta", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12158"),
     ("52_Narino", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12159"),
     ("54_Norte de Santander", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12160"),
     ("63_Quindio", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12161"),
     ("66_Risaralda", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12162"),
     ("68_Santander", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12163"),
     ("70_Sucre", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12164"),
     ("73_Tolima", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12165"),
     ("76_Valle del Cauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12166"),
     ("81_Arauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12167"),
     ("85_Casanare", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12168"),
     ("86_Putumayo", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12169"),
     ("88_San Andres Providencia y SantaCatalina", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12170"),
     ("91_Amazonas", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12171"),
     ("94_Guainia", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12172"),
     ("95_Guaviare", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12173"),
     ("97_Vaupes", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12174"),
     ("99_Vichada", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12175"),
 ]

# Procesar la información de viviendas (file_prefix es "CNPV2018_2HOG")
censo_viviendas_2018_dane = process_multiple_departments(departments_urls, "CNPV2018_1VIV_A2")


- Tamaño del dataset obtenido

In [ ]:
censo_viviendas_2018_dane.shape

(16080499, 30)

- Estructura del dataset creado a partir de las descargas

In [ ]:
censo_viviendas_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080499 entries, 0 to 16080498
Data columns (total 30 columns):
 #   Column             Dtype  
---  ------             -----  
 0   TIPO_REG           int64  
 1   U_DPTO             int64  
 2   U_MPIO             int64  
 3   UA_CLASE           int64  
 4   U_EDIFICA          int64  
 5   COD_ENCUESTAS      int64  
 6   U_VIVIENDA         int64  
 7   UVA_ESTATER        float64
 8   UVA1_TIPOTER       float64
 9   UVA2_CODTER        float64
 10  UVA_ESTA_AREAPROT  int64  
 11  UVA1_COD_AREAPROT  float64
 12  UVA_USO_UNIDAD     int64  
 13  V_TIPO_VIV         float64
 14  V_CON_OCUP         float64
 15  V_TOT_HOG          float64
 16  V_MAT_PARED        float64
 17  V_MAT_PISO         float64
 18  VA_EE              float64
 19  VA1_ESTRATO        float64
 20  VB_ACU             float64
 21  VC_ALC             float64
 22  VD_GAS             float64
 23  VE_RECBAS          float64
 24  VE1_QSEM           float64
 25  VF_INTERNET     

- Analisis de las columnas para determinar su importancia para el proyecto

In [ ]:
columns = censo_viviendas_2018_dane.columns
columns

Index(['TIPO_REG', 'U_DPTO', 'U_MPIO', 'UA_CLASE', 'U_EDIFICA',
       'COD_ENCUESTAS', 'U_VIVIENDA', 'UVA_ESTATER', 'UVA1_TIPOTER',
       'UVA2_CODTER', 'UVA_ESTA_AREAPROT', 'UVA1_COD_AREAPROT',
       'UVA_USO_UNIDAD', 'V_TIPO_VIV', 'V_CON_OCUP', 'V_TOT_HOG',
       'V_MAT_PARED', 'V_MAT_PISO', 'VA_EE', 'VA1_ESTRATO', 'VB_ACU', 'VC_ALC',
       'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'VF_INTERNET', 'V_TIPO_SERSA',
       'L_TIPO_INST', 'L_EXISTEHOG', 'L_TOT_PERL'],
      dtype='object')

In [ ]:
# Significado de columna según el diccionario de datos de los dataset
meaning_columns= {'TIPO_REG': 'Tipo de registro','U_DPTO': 'Departamento', 'U_MPIO': 'Municipio','UA_CLASE': 'Clase','U_EDIFICA': 'Numero de orden de la edificacion','COD_ENCUESTAS': 'Codigo encuenta',
                  'U_VIVIENDA': 'Numero de orden en la vivienda', 'UVA_ESTATER:': 'Vivienda en una territoriedad etnica', 'UVA1_TIPOTER': 'Tipo de terrritoriedad etnica',
                  'UVA2_CODTER': 'Codigo de territoriedad etnica','UVA_ESTA_AREAPROT': 'Vivienda en area protegida', 'UVA1_COD_AREAPROT': 'Codiga del area protegida',
                  'UVA_USO_UNIDAD': 'Uso de la unidad','V_TIPO_VIV': 'Tipo de vivienda','V_CON_OCUP': 'Condicion de ocupacion','V_TOT_HOG': 'Total de hogares en la vivienda',
                  'V_MAT_PARED': 'Material predominante en las paredes exteriores','V_MAT_PISO': 'Material predominante en los pisos','VA_EE': 'Cuenta con servicios de energia electrica',
                  'VA1_ESTRATO': 'Estrato de la vivienda','VB_ACU': 'Cuenta con servicio de acueducto','VC_ALC': 'Cuenta con servicio de alcantarillado','VD_GAS': 'Cuenta con servicio de gas natural conectado a la red publica',
                  'VE_RECBAS': 'Cuenta con servicios de recoleccion de basuras','VE1_QSEM': 'Cuantas veces por semana recogen la basura','VF_INTERNET': 'Cuenta con servicio de Internet (fijo o moveil)',
                  'V_TIPO_SERSA': 'Tipo de servicio sanitario(inodoro)','L_TIPO_INST': 'LEA_tipo de institucion o establecimiento','L_EXISTEHOG': 'LEA_existencia hogar','L_TOT_PERL': 'LEA_total residentes(no pertenecen a hogares)'}
for col in meaning_columns:
  print(f'{col}: {meaning_columns[col]}')

TIPO_REG: Tipo de registro
U_DPTO: Departamento
U_MPIO: Municipio
UA_CLASE: Clase
U_EDIFICA: Numero de orden de la edificacion
COD_ENCUESTAS: Codigo encuenta
U_VIVIENDA: Numero de orden en la vivienda
UVA_ESTATER:: Vivienda en una territoriedad etnica
UVA1_TIPOTER: Tipo de terrritoriedad etnica
UVA2_CODTER: Codigo de territoriedad etnica
UVA_ESTA_AREAPROT: Vivienda en area protegida
UVA1_COD_AREAPROT: Codiga del area protegida
UVA_USO_UNIDAD: Uso de la unidad
V_TIPO_VIV: Tipo de vivienda
V_CON_OCUP: Condicion de ocupacion
V_TOT_HOG: Total de hogares en la vivienda
V_MAT_PARED: Material predominante en las paredes exteriores
V_MAT_PISO: Material predominante en los pisos
VA_EE: Cuenta con servicios de energia electrica
VA1_ESTRATO: Estrato de la vivienda
VB_ACU: Cuenta con servicio de acueducto
VC_ALC: Cuenta con servicio de alcantarillado
VD_GAS: Cuenta con servicio de gas natural conectado a la red publica
VE_RECBAS: Cuenta con servicios de recoleccion de basuras
VE1_QSEM: Cuantas vec

In [ ]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {censo_viviendas_2018_dane[col].nunique()}')

TIPO_REG: 1
U_DPTO: 33
U_MPIO: 581
UA_CLASE: 3
U_EDIFICA: 2683
COD_ENCUESTAS: 16080333
U_VIVIENDA: 1782
UVA_ESTATER: 2
UVA1_TIPOTER: 2
UVA2_CODTER: 963
UVA_ESTA_AREAPROT: 2
UVA1_COD_AREAPROT: 294
UVA_USO_UNIDAD: 3
V_TIPO_VIV: 6
V_CON_OCUP: 4
V_TOT_HOG: 19
V_MAT_PARED: 9
V_MAT_PISO: 6
VA_EE: 2
VA1_ESTRATO: 8
VB_ACU: 2
VC_ALC: 2
VD_GAS: 3
VE_RECBAS: 2
VE1_QSEM: 9
VF_INTERNET: 3
V_TIPO_SERSA: 7
L_TIPO_INST: 13
L_EXISTEHOG: 3
L_TOT_PERL: 403


In [ ]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in columns:
  l = censo_viviendas_2018_dane[col].nunique()
  if l < 50:
    print(f'{col}: {censo_viviendas_2018_dane[col].unique()}')

TIPO_REG: [1]
U_DPTO: [ 5  8 11 13 15 17 18 19 20 23 25 27 41 44 47 50 52 54 63 66 68 70 73 76
 81 85 86 88 91 94 95 97 99]
UA_CLASE: [1 2 3]
UVA_ESTATER: [nan  2.  1.]
UVA1_TIPOTER: [nan  1.  2.]
UVA_ESTA_AREAPROT: [2 1]
UVA_USO_UNIDAD: [1 2 4]
V_TIPO_VIV: [ 1.  2.  3.  6.  5.  4. nan]
V_CON_OCUP: [ 1.  4.  2.  3. nan]
V_TOT_HOG: [ 2.  1.  6.  3. nan  4.  5.  8.  7.  9. 10. 11. 12. 14. 15. 18. 17. 13.
 20. 16.]
V_MAT_PARED: [ 1.  8.  2. nan  5.  3.  6.  7.  4.  9.]
V_MAT_PISO: [ 2.  5.  4. nan  1.  3.  6.]
VA_EE: [ 1. nan  2.]
VA1_ESTRATO: [ 2.  1. nan  3.  4.  0.  5.  6.  9.]
VB_ACU: [ 1. nan  2.]
VC_ALC: [ 1. nan  2.]
VD_GAS: [ 1.  2. nan  9.]
VE_RECBAS: [ 1. nan  2.]
VE1_QSEM: [ 2.  4. nan  3.  1.  7.  9.  5.  8.  6.]
VF_INTERNET: [ 2.  1. nan  9.]
V_TIPO_SERSA: [ 1. nan  5.  3.  2.  9.  6.  4.]
L_TIPO_INST: [nan  3.  2. 12.  4.  1.  5.  6.  8. 10.  7.  9. 13. 11.]
L_EXISTEHOG: [nan  1.  2.  9.]


In [ ]:
# Eliminar columnas que se considera innecesarias para el proyecto
important_columns = ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'UVA_ESTATER', 'UVA1_TIPOTER', 'UVA_ESTA_AREAPROT','UVA_USO_UNIDAD', 'V_TIPO_VIV', 'V_CON_OCUP', 'V_TOT_HOG',
       'V_MAT_PARED', 'V_MAT_PISO', 'VA_EE', 'VA1_ESTRATO', 'VB_ACU', 'VC_ALC', 'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'VF_INTERNET', 'V_TIPO_SERSA','L_TOT_PERL']
censo_viviendas_2018_dane = censo_viviendas_2018_dane[important_columns]
censo_viviendas_2018_dane.head()

,U_DPTO,U_MPIO,UA_CLASE,UVA_ESTATER,UVA1_TIPOTER,UVA_ESTA_AREAPROT,UVA_USO_UNIDAD,V_TIPO_VIV,V_CON_OCUP,V_TOT_HOG,...,VA_EE,VA1_ESTRATO,VB_ACU,VC_ALC,VD_GAS,VE_RECBAS,VE1_QSEM,VF_INTERNET,V_TIPO_SERSA,L_TOT_PERL
0,5,1,1,NaN,NaN,2,1,1.0,1.0,2.0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,NaN
1,5,1,1,NaN,NaN,2,1,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN
2,5,1,1,NaN,NaN,2,1,2.0,1.0,6.0,...,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN
3,5,1,1,NaN,NaN,2,1,1.0,1.0,3.0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN
4,5,1,1,NaN,NaN,2,1,2.0,1.0,1.0,...,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN


In [ ]:
censo_viviendas_2018_dane.shape

(16080499, 22)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios y los departamentos están almacenados por aparte y en formato int64


In [ ]:
# valores codigo departamento
print(f'Valores unicos codigos depto: {censo_viviendas_2018_dane["U_DPTO"].unique()}')
print(f'Valores unicos codigos mpios: {censo_viviendas_2018_dane["U_MPIO"].unique()}')

Valores unicos codigos depto: [ 5  8 11 13 15 17 18 19 20 23 25 27 41 44 47 50 52 54 63 66 68 70 73 76
 81 85 86 88 91 94 95 97 99]
Valores unicos codigos mpios: [  1   2   4  21  30  31  34  36  38  40  42  44  45  51  55  59  79  86
  88  91  93 101 107 113 120 125 129 134 138 142 145 147 148 150 154 172
 190 197 206 209 212 234 237 240 250 264 266 282 284 306 308 310 313 315
 318 321 347 353 360 361 364 368 376 380 390 400 411 425 440 467 475 480
 483 490 495 501 541 543 576 579 585 591 604 607 615 628 631 642 647 649
 652 656 658 659 660 664 665 667 670 674 679 686 690 697 736 756 761 789
 790 792 809 819 837 842 847 854 856 858 861 873 885 887 890 893 895  78
 137 141 296 372 421 433 436 520 549 558 560 573 606 634 638 675 685 758
 770 832 849   6  52  62  74 140 160 188 222 244 248 268 300 430 442 458
 468 473 580 600 620 650 654 655 657 673 683 688 744 760 780 810 836 838
 894  22  47  87  90  92  97 104 106 109 114 131 135 162 176 180 183 185
 187 189 204 215 218 223 224 226 23

- Longitud de los códigos de departamentos y municipios en el df

In [ ]:
# Convertir a strings
censo_viviendas_2018_dane[['U_DPTO','U_MPIO']] = censo_viviendas_2018_dane[['U_DPTO','U_MPIO']].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_depto = censo_viviendas_2018_dane['U_DPTO'].apply(len)
longitud_mpio = censo_viviendas_2018_dane['U_MPIO'].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los dptos tiene longitudes de :{longitud_depto.unique()} caracteres')
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los dptos tiene longitudes de :[1 2] caracteres
Los códigos de los mpios tiene longitudes de :[1 2 3] caracteres


- Convertir los códigos de los departamentos en un string de 2 digitos y los de los municipios en un string de 3 digitos para que coincidan con el formato del código almacenado en la base de datos

Rellenamos con ceros a la izquierda aquellos códigos del departamento que solo tienen un dígito y aquellos códigos de los municipios que solo tienen 1 y 2 dígitos


In [ ]:
# Función para formatear los códigos
def formatear_codigo(codigo, longitud):
    return codigo.zfill(longitud)

# Aplicar la función a las columnas y crear la nueva columna
censo_viviendas_2018_dane['U_DPTO'] = censo_viviendas_2018_dane['U_DPTO'].apply(formatear_codigo, args=(2,))
censo_viviendas_2018_dane['U_MPIO'] = censo_viviendas_2018_dane['U_MPIO'].apply(formatear_codigo, args=(3,))

- Crear nueva columna que contenga el codigo del departamento mas el codigo del municipio

In [ ]:
# Nueva columna de código de municipio con longitud de 5 caracteres
censo_viviendas_2018_dane['COD_DEP_MPIO'] = censo_viviendas_2018_dane['U_DPTO'] +censo_viviendas_2018_dane['U_MPIO']

# Verificar
censo_viviendas_2018_dane['COD_DEP_MPIO']

,COD_DEP_MPIO
0,05001
1,05001
2,05001
3,05001
4,05001
...,...
16080494,99773
16080495,99773
16080496,99773
16080497,99773


In [ ]:
# Eliminar columnas 'U_DPTO' y 'U_MPIO' que ya no son necesarias
censo_viviendas_2018_dane = censo_viviendas_2018_dane.drop(['U_DPTO', 'U_MPIO'], axis=1)
censo_viviendas_2018_dane.head(3)

,UA_CLASE,UVA_ESTATER,UVA1_TIPOTER,UVA_ESTA_AREAPROT,UVA_USO_UNIDAD,V_TIPO_VIV,V_CON_OCUP,V_TOT_HOG,V_MAT_PARED,V_MAT_PISO,...,VA1_ESTRATO,VB_ACU,VC_ALC,VD_GAS,VE_RECBAS,VE1_QSEM,VF_INTERNET,V_TIPO_SERSA,L_TOT_PERL,COD_DEP_MPIO
0,1,NaN,NaN,2,1,1.0,1.0,2.0,1.0,2.0,...,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,NaN,05001
1,1,NaN,NaN,2,1,1.0,1.0,1.0,1.0,2.0,...,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,05001
2,1,NaN,NaN,2,1,2.0,1.0,6.0,1.0,5.0,...,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,05001


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [ ]:
# Comparar listas de códigos
compare_lists(censo_viviendas_2018_dane['COD_DEP_MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_by_type_of_crime_UnidadVictimas", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_by_type_of_crime_UnidadVictimas que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


Nota: El codigo 94663 pertence a Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio Barranco Minas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [ ]:
censo_viviendas_2018_dane['COD_DEP_MPIO'] = censo_viviendas_2018_dane['COD_DEP_MPIO'].replace( '94663', '94343')

- Volvemos a veridfcar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [ ]:
# Comparar listas de códigos
compare_lists(censo_viviendas_2018_dane['COD_DEP_MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_by_type_of_crime_UnidadVictimas", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_by_type_of_crime_UnidadVictimas que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Adicionar columna para trazabilidad de la fuente
censo_viviendas_2018_dane['SOURCE_ID'] = 55

In [ ]:
censo_viviendas_2018_dane.columns

Index(['UA_CLASE', 'UVA_ESTATER', 'UVA1_TIPOTER', 'UVA_ESTA_AREAPROT',
       'UVA_USO_UNIDAD', 'V_TIPO_VIV', 'V_CON_OCUP', 'V_TOT_HOG',
       'V_MAT_PARED', 'V_MAT_PISO', 'VA_EE', 'VA1_ESTRATO', 'VB_ACU', 'VC_ALC',
       'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'VF_INTERNET', 'V_TIPO_SERSA',
       'L_TOT_PERL', 'COD_DEP_MPIO', 'SOURCE_ID'],
      dtype='object')

In [ ]:
#Estructura final del dataset a integrar a la base de datos
censo_viviendas_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080499 entries, 0 to 16080498
Data columns (total 22 columns):
 #   Column             Dtype  
---  ------             -----  
 0   UA_CLASE           int64  
 1   UVA_ESTATER        float64
 2   UVA1_TIPOTER       float64
 3   UVA_ESTA_AREAPROT  int64  
 4   UVA_USO_UNIDAD     int64  
 5   V_TIPO_VIV         float64
 6   V_CON_OCUP         float64
 7   V_TOT_HOG          float64
 8   V_MAT_PARED        float64
 9   V_MAT_PISO         float64
 10  VA_EE              float64
 11  VA1_ESTRATO        float64
 12  VB_ACU             float64
 13  VC_ALC             float64
 14  VD_GAS             float64
 15  VE_RECBAS          float64
 16  VE1_QSEM           float64
 17  VF_INTERNET        float64
 18  V_TIPO_SERSA       float64
 19  L_TOT_PERL         float64
 20  COD_DEP_MPIO       object 
 21  SOURCE_ID          int64  
dtypes: float64(17), int64(4), object(1)
memory usage: 2.6+ GB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
censo_viviendas_2018_dane.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/censo_viviendas_2018_dane.csv', index=False)